Country by commodity exports

In [1]:
from gssutils import *

scraper = Scraper('https://www.ons.gov.uk/economy/nationalaccounts/balanceofpayments/datasets/' + \
                 'uktradecountrybycommodityexports')
scraper

## Trade in goods: country-by-commodity exports

Monthly export country-by-commodity data on the UK's trade in goods, including trade by all countries and selected commodities, non-seasonally adjusted.

### Distributions

1. Trade in goods: country-by-commodity exports ([MS Excel Spreadsheet](https://www.ons.gov.uk/file?uri=/economy/nationalaccounts/balanceofpayments/datasets/uktradecountrybycommodityexports/current/countrybycommodityexportsfinal.xlsx))


In [2]:
tab = scraper.distribution().as_pandas()
tab

,COMMODITY,COUNTRY,DIRECTION,1998JAN,1998FEB,1998MAR,1998APR,1998MAY,1998JUN,1998JUL,...,2018JAN,2018FEB,2018MAR,2018APR,2018MAY,2018JUN,2018JUL,2018AUG,2018SEP,2018OCT
0,0 Food & live animals,AD Andorra,EX Exports,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0 Food & live animals,AE United Arab Emirates,EX Exports,5,5,5,4,5,4,6,...,19,17,21,21,19,17,20,30,19,20
2,0 Food & live animals,AF Afghanistan,EX Exports,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0 Food & live animals,AG Antigua & Barbuda,EX Exports,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0 Food & live animals,AI Anguilla,EX Exports,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0 Food & live animals,AL Albania,EX Exports,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0 Food & live animals,AM Armenia,EX Exports,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0 Food & live animals,AO Angola,EX Exports,0,0,0,0,0,0,0,...,0,0,2,1,0,1,2,1,1,3
8,0 Food & live animals,AQ Antarctica,EX Exports,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0 Food & live animals,AR Argentina,EX Exports,0,0,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [3]:
tab = tab.drop(['DIRECTION'], axis = 1)

In [4]:
tab.columns.values[0] = 'CORD SITC'
tab.columns.values[1] = 'ONS Partner Geography'

In [5]:
new_table = pd.melt(tab, id_vars=['CORD SITC','ONS Partner Geography'], var_name='Period', value_name='Value')

In [6]:
import re
YEAR_RE = re.compile(r'[0-9]{4}')
YEAR_MONTH_RE = re.compile(r'([0-9]{4})(JAN|FEB|MAR|APR|MAY|JUN|JUL|AUG|SEP|OCT|NOV|DEC)')
YEAR_QUARTER_RE = re.compile(r'([0-9]{4})\s+(Q[1-4])')

# from https://stackoverflow.com/questions/597476/how-to-concisely-cascade-through-multiple-regex-statements-in-python
class Re(object):
  def __init__(self):
    self.last_match = None
  def fullmatch(self,pattern,text):
    self.last_match = re.fullmatch(pattern,text)
    return self.last_match

def time2period(t):
    gre = Re()
    if gre.fullmatch(YEAR_RE, t):
        return f"year/{t}"
    elif gre.fullmatch(YEAR_MONTH_RE, t):
        year, month = gre.last_match.groups()
        month_num = {'JAN': '01', 'FEB': '02', 'MAR': '03', 'APR': '04', 'MAY': '05', 'JUN': '06',
                     'JUL': '07', 'AUG': '08', 'SEP': '09', 'OCT': '10', 'NOV': '11', 'DEC': '12'}.get(month)
        return f"month/{year}-{month_num}"
    elif gre.fullmatch(YEAR_QUARTER_RE, t):
        year, quarter = gre.last_match.groups()
        return f"quarter/{year}-{quarter}"
    else:
        print(f"no match for {t}")

new_table['Period'] = new_table['Period'].apply(time2period)

In [7]:
new_table['Seasonal Adjustment'] =  'NSA'
new_table['Measure Type'] =  'GBP Total'
new_table['Unit'] =  'gbp-million'
new_table['Flow'] = 'exports'

Todo: check following logic (see imports)

In [8]:
# new_table['Value'] = pd.to_numeric(new_table['Value'], errors='coerce').fillna(0)

In [9]:
new_table['Value'] = new_table['Value'].astype(int)

In [10]:
new_table['ONS Partner Geography'] = new_table['ONS Partner Geography'].astype(str).str[0:2] 

In [11]:
new_table['CORD SITC'] = new_table['CORD SITC'].str.partition(' ')

Todo: check following logic (see imports)

In [12]:
# new_table = new_table[new_table['Value'] !=  0 ]

In [13]:
new_table = new_table[['ONS Partner Geography', 'Period','Flow','CORD SITC', 'Seasonal Adjustment', 'Measure Type','Value','Unit' ]]

In [14]:
new_table.count()

ONS Partner Geography    7406250
Period                   7406250
Flow                     7406250
CORD SITC                7406250
Seasonal Adjustment      7406250
Measure Type             7406250
Value                    7406250
Unit                     7406250
dtype: int64

In [15]:
new_table.dtypes

ONS Partner Geography    object
Period                   object
Flow                     object
CORD SITC                object
Seasonal Adjustment      object
Measure Type             object
Value                     int32
Unit                     object
dtype: object